- sampled MimicIV discharge notes 10000
- using TF-Idf
- Remove useless words : "discharge"

In [2]:
import pickle
import pandas as pd
import json

from utils import *
config = load_config()

In [6]:
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")

tfidf_data = pd.read_pickle(DATA_PATH.joinpath("tfidf_jargons.pkl"))
filtered_discharge_dataset = pd.read_pickle(DATA_PATH.joinpath('filtered_discharge_dataset.pkl'))

In [7]:
len(tfidf_data), len(filtered_discharge_dataset)

(10000, 10000)

In [8]:
# parse tfidf data
dict(tfidf_data[0])

{'excess fluid': 0.8767082464408509,
 'Medications': 0.5229258696875738,
 'Discharge': 1.8113426903876666,
 'schedule': 0.11566176734975514,
 'likely': 0.19746992261217688,
 'lasix': 0.2646661024475417,
 'liver': 0.5931368186300708,
 'pain': 0.2978813783058901,
 'care': 0.1988363993894176}

In [9]:
for key, val in dict(sorted(tfidf_data[0].items(), key=lambda item : item[1], reverse=True)).items() :
    print(key, val)

Discharge 1.8113426903876666
excess fluid 0.8767082464408509
liver 0.5931368186300708
Medications 0.5229258696875738
pain 0.2978813783058901
lasix 0.2646661024475417
care 0.1988363993894176
likely 0.19746992261217688
schedule 0.11566176734975514


In [70]:
print(filtered_discharge_dataset.iloc[0])

note_id                                                      10000032-DS-21
subject_id                                                         10000032
hadm_id                                                            22595853
note_type                                                                DS
note_seq                                                                 21
charttime                                               2180-05-07 00:00:00
storetime                                               2180-05-09 15:26:00
text                       \nName:  ___                     Unit No:   _...
front_text                 \nName:  ___                     Unit No:   _...
discharge_instructions    Discharge Instructions:\nDear Ms. ___,\nIt was...
Name: 0, dtype: object


In [10]:
def parse_dataset_into_json(row_n) :
    text = filtered_discharge_dataset.iloc[row_n]['text']
    keywords = dict(sorted(tfidf_data[row_n].items(), key=lambda item : item[1], reverse=True))
    output = ""
    for i, (key,val) in enumerate(keywords.items()) :
        i += 1 # starting from 1
        output += f"{i}. {key.lower()}\n"

    with open(PROJECT_PATH.joinpath("prompts/finetune_instruction.txt")) as f :
        instruction = f.read()

    formated_terms = {"input" : text,
                    "output" : output, 
                    "instruction" : instruction}

    return formated_terms

In [11]:
parsed_dataset = []
for idx in range(len(filtered_discharge_dataset)) : 
    output = parse_dataset_into_json(idx)
    parsed_dataset.append(output)

In [17]:
text = parsed_dataset[0]['instruction']
print(text)

You are a helpful assistant, an expert in medical domain. 
Extract top {topN} main diagnosis/symptoms or conditions mentioned in the medical note. 
Following the diagnosis/symptoms or conditions, identify the medical tests related to it.
If there isn't any medical tests related to it, just start listing the next important diagnosis/symptoms or conditions.
If there are no additional diagnosis/symptoms or conditions that you can identify, just list the existing ones and finalize the output. 
Don't write no symptoms, or any indication that there is no other diagnosis/symptoms or conditions.
Do not modify or abbreviate what is written in the notes. Just extract them as they are.
Make sure the highest priority is assigned with a smaller number.
We give you an example, do follow as below.
The format should be as follows

1. key symptom, lab tests, condition
2. key symptom, lab tests, condition
3. key symptom, lab tests, condition
4. key symptom, lab tests, condition
5. key symptom, lab tests

In [45]:
import json
with open(DATA_PATH.joinpath("discharge_dataset.json"), "w") as f : 
    json.dump(parsed_dataset, f)

In [96]:
import random
random.seed(42)

with open(DATA_PATH.joinpath("discharge_dataset.json"), "r") as f :
    parsed_dataset = json.load(f)

random.shuffle(parsed_dataset)

In [99]:
idx = int(len(parsed_dataset)*0.2)
testset, trainset = parsed_dataset[:idx], parsed_dataset[idx:]

In [102]:
len(testset), len(trainset)

(2000, 8000)

In [103]:
with open(DATA_PATH.joinpath("discharge_dataset_train.json"), "w") as f : 
    json.dump(trainset, f)

with open(DATA_PATH.joinpath("discharge_dataset_test.json"), "w") as f : 
    json.dump(testset, f)